In [1]:
pip install pulp cplex scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 13.9 MB/s eta 0:00:00


The general form of LP problems we will consider is:

        min_x  cᵀx
   subject to  Gx ≥ h
               Ax = b
               l ≤ x ≤ u

The benchmark datasets use the .mps format for LP problems so there is a function to extract the parameters from this file type into the form above. But since I haven't figured out how to access these datasets yet there is also a function to generate large feasible LP example problems in the .mps format. Finally, there is a preliminary function that implements the PDHG algorithm but not yet any of the enhancments.

In [6]:
import numpy as np
from scipy.sparse import random as sparse_random
import pulp

def generate_feasible_lp(num_vars=100, num_ineq=200, num_eq=50, density=0.05, mps_filename="generated_lp.mps"):
  '''This function generates large feasible LP problems to test and saves them in the .mps format'''

  # The number of variables in each constraint with nonzero coefficients will be roughly density * num_vars

  rng = np.random.default_rng(0) #Random number generator

  # Step 1: (Random) Feasible solution
  x_feas = rng.uniform(low=-10, high=10, size=(num_vars, 1))

  # Step 2: Sparse matrices (convert to dense)
  G_sparse = sparse_random(num_ineq, num_vars, density=density, format='csr', random_state=None)
  A_sparse = sparse_random(num_eq, num_vars, density=density, format='csr', random_state=None)

  G = G_sparse.toarray()
  A = A_sparse.toarray()

  # Step 3: RHS vectors
  h = G @ x_feas #+ rng.uniform(0.1, 5.0, size=(num_ineq, 1))
  b = A @ x_feas

  # Step 4: Bounds
  l = x_feas - rng.uniform(1, 5, size=(num_vars, 1))
  u = x_feas + rng.uniform(1, 5, size=(num_vars, 1))
  l = np.maximum(l, -1e4)
  u = np.minimum(u, 1e4)

  # Step 5: Objective
  c = rng.normal(size=(num_vars, 1))

  # Step 6: Write to MPS using pulp
  prob = pulp.LpProblem("Feasible_LP", pulp.LpMinimize)
  x_vars = [
      pulp.LpVariable(f"x{i}", lowBound=float(l[i]), upBound=float(u[i]))
      for i in range(num_vars)
  ]
  prob += pulp.lpDot(c.flatten(), x_vars)

  # Inequality constraints: Gx <= h
  for i in range(num_ineq):
      prob += pulp.lpDot(G[i], x_vars) <= float(h[i]), f"ineq_{i}"

  # Equality constraints: Ax = b
  for i in range(num_eq):
      prob += pulp.lpDot(A[i], x_vars) == float(b[i]), f"eq_{i}"

  prob.writeMPS(mps_filename)
  print(f"LP written to: {mps_filename}")

In [9]:
import numpy as np
import cplex
from cplex.exceptions import CplexError

def mps_to_standard_form(mps_file):
  '''
  Extracts the parameters of the LP from the .mps format to the general form
  Returns c, G ineq matrix, h ineq vec, A eq matrix, b eq vec, l and u lower and
  upper box constraints
  '''
  try:
      cpx = cplex.Cplex(mps_file)
      cpx.set_results_stream(None)  # mute output

      # Number of variables and constraints
      num_vars = cpx.variables.get_num()
      num_constraints = cpx.linear_constraints.get_num()

      # Objective vector (c)
      c = np.array(cpx.objective.get_linear())

      # Constraint matrix
      A_full = cpx.linear_constraints.get_rows()
      senses = cpx.linear_constraints.get_senses()
      rhs = np.array(cpx.linear_constraints.get_rhs())

      A = []
      G = []
      b = []
      h = []

      for i, (row, sense, rhs_i) in enumerate(zip(A_full, senses, rhs)):
          row_vec = np.zeros(num_vars)
          for idx, val in zip(row.ind, row.val):
              row_vec[idx] = val
          if sense == 'E':  # Equality constraint
              A.append(row_vec)
              b.append(rhs_i)
          elif sense == 'G':  # Greater than or equal
              G.append(row_vec)
              h.append(rhs_i)
          elif sense == 'L':  # Less than or equal
              # convert to -Gx ≥ -h
              G.append(-row_vec)
              h.append(-rhs_i)

      A = np.array(A) if A else np.zeros((0, num_vars))
      b = np.array(b) if b else np.zeros(0)
      G = np.array(G) if G else np.zeros((0, num_vars))
      h = np.array(h) if h else np.zeros(0)

      # Bounds
      l = np.array(cpx.variables.get_lower_bounds())
      u = np.array(cpx.variables.get_upper_bounds())

      c = c.reshape(-1, 1)
      h = h.reshape(-1, 1)
      b = b.reshape(-1, 1)
      l = l.reshape(-1, 1)
      u = u.reshape(-1, 1)

      return c, G, h, A, b, l, u

  except CplexError as e:
      print("CPLEX Error:", e)
      return None

In [10]:
import numpy as np

def proj_Lam(lam):
    return lam

def pdhg(c, G, h, A, b, l, u, max_iter=1000, tol=1e-4, term_period=1000):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using PDHG algorithm.
    """

    # Define Parameters
    K = np.vstack([G, A])
    q = np.vstack([h, b])

    eta = 0.9/np.linalg.norm(K, 2)
    omega = 10

    tau = eta/omega
    sigma = eta*omega

    m_1 = G.shape[0]
    m_2 = A.shape[0]
    n = c.shape[0]

    # Termination Parameters
    tol_dual = tol * (1 + np.linalg.norm(c))
    tol_prim = tol * (1 + np.linalg.norm(q))

    # Initialize Primal and Dual Variables
    x = np.minimum(np.maximum(np.zeros((n, 1)), l), u)
    y = np.zeros((m_1 + m_2, 1))

    for i in range(1, max_iter + 1):
        # Primal update
        x_old = x.copy()
        # Project x onto box constraints l ≤ x ≤ u
        grad = c - K.T @ y
        x = np.minimum(np.maximum(x - tau * grad, l), u)

        # Dual update
        y += sigma * (q - K @ (2*x - x_old))
        y[:m_1] = np.maximum(y[:m_1], 0)  # project onto constraint y:m_1 ≥ 0

        # Check Termination Criteria Periodically
        if i%term_period == 0:

            dual_op = (q.T @ y)[0][0]
            prim_op = (c.T @ x)[0][0]

            lam_p_op = (l.T @ np.maximum(grad, 0))[0][0]
            lam_n_op = (u.T @ np.minimum(grad, 0))[0][0]

            #print(dual_op + lam_p_op + lam_n_op, prim_op)
            #print(np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])))
            #print(np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])), np.linalg.norm(grad - proj_Lam(grad)), np.abs(dual_op + lam_p_op - lam_n_op - prim_op))
            #print(tol_dual, tol_prim, tol * (1 + np.abs(dual_op + lam_p_op - lam_n_op) + np.abs(prim_op)))

            #Termination criteria
            if (
                np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])) <= tol_prim
                and np.linalg.norm(grad - proj_Lam(grad)) <= tol_dual
                and np.abs(dual_op + lam_p_op + lam_n_op - prim_op) <= tol * (1 + np.abs(dual_op + lam_p_op - lam_n_op) + np.abs(prim_op))
            ):
                break

    # Returns minimal objective value, minimizer estimate in list format, and the number of iterations run
    return (c.T @ x)[0][0], x.T[0].tolist(), i

Testing the algorithm

In [11]:
from time import perf_counter

# Makes a timer to measure code omtimality
class Timer:
    def __enter__(self):
        self.start = perf_counter()
        return self
    def __exit__(self, *args):
        self.end = perf_counter()
        self.elapsed = self.end - self.start
        print(f"Elapsed time: {self.elapsed:.6f} seconds")


In [ ]:
# Create a feasible LP problem and save to a
generate_feasible_lp(num_vars=1000, num_ineq=500, num_eq=500, density=0.05, mps_filename="large_example.mps")

/tmp/ipykernel_147092/277239451.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pulp.LpVariable(f"x{i}", lowBound=float(l[i]), upBound=float(u[i]))
/tmp/ipykernel_147092/277239451.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob += pulp.lpDot(G[i], x_vars) <= float(h[i]), f"ineq_{i}"
/tmp/ipykernel_147092/277239451.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob += pulp.lpDot(A[i], x_vars) == float(b[i]), f"eq_{i}"


LP written to: large_example.mps


In [12]:
import cplex

# Solve the LP using either primal simplex, dual simplex, or barrier (interior point).
# Only works for LP with no more than 1000 constraints and no more than 1000 variables
cpx = cplex.Cplex("large_example.mps")
c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")

# Times how long it takes to solve
with Timer():
    cpx.solve()

# Save the minimizer and minimal objective values for comparison
cpx_obj_val = cpx.solution.get_objective_value()
cpx_min = cpx.solution.get_values()
print("Objective value:", cpx_obj_val)
#print("Minimizer: xᵀ =", cpx_min)

CPLEX Error  1423: Could not open file 'large_example.mps' for reading.
CPLEX Error  1423: Could not open file 'large_example.mps.sav' for reading.
CPLEX Error  1423: Could not open file 'large_example.mps' for reading.
CPLEX Error  1423: Could not open file 'large_example.mps' for reading.


CplexSolverError: CPLEX Error  1423: Could not open file 'large_example.mps' for reading.

In [13]:
# Extract LP parameters from generated example
with Timer():
    c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")
    pdhg_obj_val, pdhg_min, iterations = pdhg(c, G, h, A, b, l, u, max_iter=1000000, tol=1e-8)
print("Objective Value:", pdhg_obj_val)
print("Iterations:", iterations)
#print("Minimizer: xᵀ =",pdhg_min)

# The distance between the two minimizer solutions
# Should be small but won't be incredibly small since the vectors are high dimensional
#distance = np.linalg.norm(np.array(pdhg_min) - np.array(cpx_min))
#print("Distance:", distance)

CPLEX Error  1423: Could not open file 'large_example.mps' for reading.
CPLEX Error  1423: Could not open file 'large_example.mps.sav' for reading.
CPLEX Error  1423: Could not open file 'large_example.mps' for reading.
CPLEX Error  1423: Could not open file 'large_example.mps' for reading.


CPLEX Error: CPLEX Error  1423: Could not open file 'large_example.mps' for reading.
Elapsed time: 0.007759 seconds


TypeError: cannot unpack non-iterable NoneType object